# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import httpx
from selectolax.parser import HTMLParser
import re

# Get Data From vlr.gg

## Data Extracting & Cleaning Function

In [2]:
def extract_df(df):
    
    
    def extract_data(data) -> str:
        list_data = re.findall("[\d+-.]+",data)

        if len(list_data) < 3:
            result = '0 0 0'
        else:
            result = ' '.join(list_data)

        return result
    
    filled_df =df.fillna('Nan Nan Nan').astype('object')
    
    name = filled_df.iloc[:,0].map(lambda s : s.split(' ')[0])
    team = filled_df.iloc[:,0].map(lambda s : s.split(' ')[1])
    
    filled_df.iloc[:,0] = name
    filled_df.iloc[:,1] = team
    
    formatted_df = filled_df.rename(columns={'Unnamed: 0':'Name','Unnamed: 1':'Team'})
    
    formatted_df.iloc[:,2:] = formatted_df.iloc[:,2:].applymap(extract_data)
    
    new_columns = []
    for c in formatted_df.columns[2:]:
        for side in ['all','atk','def']:
            new_columns.append((c,side))

    new_columns = np.array(new_columns).reshape(-1,3,2)
    
    
    result_df = pd.DataFrame(formatted_df.iloc[:,:2])
    
    
    result_df.columns = [('Name','Name'),('Team','Team')]
    
    
    for nc,c in zip(new_columns,formatted_df.iloc[:,2:].columns):
        result_df[list(map(lambda x: tuple(x),list(nc)))] = formatted_df[c].str.split(' ',expand=True)
        
    
    result_df.columns = pd.MultiIndex.from_tuples(result_df.columns,name=['Type','Side'])
    
    return result_df

## Scraping Data

### Preparing URL(s)

In [3]:
base_url = 'https://www.vlr.gg'

event_url = '/event/matches/1188/champions-tour-2023-lock-in-s-o-paulo/?series_id=all'

### Scraping All Matches in the Event

In [4]:
res = httpx.get(base_url+event_url) #get https response from vlr.gg

html = HTMLParser(res.text) # parsing into plain text

matches = html.css('a.wf-module-item') # get all matches in events

# find all completed match by css status class
completed_matches = [match for match in matches \
                     if match.css_first('div.ml-status').text() == 'Completed'] 

In [5]:
# incase need a perfect matches with no missing data point(s) for VCT LOCK//IN
# perfect_completed_matches = completed_matches[:21] + completed_matches[22:30] + completed_matches[31:]

### Scraping Map(s) played and players data

In [64]:
re_strip = lambda sp,st : sp.join(re.findall('\S+',st)) # function for normal regex by finding all char

for index,match in enumerate(completed_matches):
    
    # formatting name (tbh i couldn't remember why i did this but for the pretty formatted name)
    match_res = httpx.get(match_url := f'{base_url}{match.attributes["href"]}')
    print(httpx.get(match_url))

    match_html = HTMLParser(match_res.text)

    match_name = match_html.css_first('title').text().strip().split(' | ')[0].split(' vs. ')

    messy_match_result = match_html.css_first('div.js-spoiler').text()
    match_result = ' ' + re_strip('',messy_match_result) + ' '

    print(index, '>> ',end='')
    print(match_sum:=match_result.join(match_name))
    print()
    
    # get date and time of the match
    date = match_html.css_first('div.match-header-date')
    print('\t'+re_strip(' ',date.text()))
    print()
    
    # stage of the match
    stage = match_html.css_first('div.match-header-event-series')
    print('\t'+re_strip(' ',stage.text()))
    print()
    
    #ban pick of the match
    ban_pick = match_html.css_first('div.match-header-note')
    for bp in re_strip(' ',ban_pick.text()).split(';'):
        print('\t'+bp.strip())
#     #scraping all tables from pages w/ css table class
#     tables = match_html.css('table.wf-table-inset')

    
#     for table in tables: # for all table in matches
#         df = pd.read_html(table.html)[0]
#         display(transform_subcolumns_df(df))


    # get data of each map(s) in the match
    maps = match_html.css('div.vm-stats-game')

    # rearrange the map order
    maps = [maps[1],maps[0],maps[2]]
    
    # extract data of each map
    for m in maps:
        # get the current map pick name or overall if it the match summary
        current_map = m.css_first('div.map')
        match_header = 'Overall' if not current_map else\
                        ' '.join(re_strip(' ',current_map.text()).split(' ')[::2])
        
        print(match_header)
        
        
        tables = m.css('table.wf-table-inset')
        
        for table in tables:
            df = pd.read_html(table.html)[0]
            display(extract_df(df))
        

<Response [200 OK]>
0 >> KOI 0:2 NRG Esports

	Tuesday, February 14th 12:10 AM +07 Patch 6.02

	Bracket Stage: Alpha - Round of 16

	NRG ban Ascent
	KOI ban Split
	NRG pick Icebox
	KOI pick Haven
	NRG ban Fracture
	KOI ban Lotus
	Pearl remains
Overall


Type        Name Team     R              ACS             K      ... HS%  FK  \
Side        Name Team   all   atk   def  all  atk  def all atk  ... def all   
0          trexx  KOI  1.24  1.45  1.03  275  329  229  46  28  ...  37   3   
1        sheydos  KOI  0.95  1.16  0.75  158  188  135  27  18  ...  25   2   
2         starxo  KOI  0.76  0.79  0.72  123  123  124  17   9  ...  13   0   
3         Wolfen  KOI  0.75  0.81  0.70  187  214  161  28  16  ...  40  10   
4     koldamenta  KOI  0.72  0.77  0.64  110  115  103  17   8  ...  20   1   

Type          FD         +/–.1          
Side atk def all atk def   all atk def  
0      2   1   4   1   3    -1  +1  -2  
1      1   1   4   1   3    -2   0  -2  
2      0   0   6   1   5    -6  -1  -5  
3      6   4  13   7   6    -3  -1  -2  
4      1   0   3   2   1    -2  -1  -1  

[5 rows x 38 columns]

Type      Name Team     R              ACS             K      ... HS%  FK      \
Side      Name Team   all   atk   def  all  atk  def all atk  ... def all atk   
0     crashies  NRG  1.26  1.44  1.08  220  252  188  36  24  ...  22   5   1   
1          s0m  NRG  1.25  1.29  1.23  227  209  253  36  17  ...  41   4   2   
2          FNS  NRG  1.12  1.15  1.10  173  161  186  24  12  ...  17   5   4   
3       ardiis  NRG  1.08  1.27  0.88  190  217  159  27  16  ...  12  10   8   
4       Victor  NRG  0.98  1.05  0.92  172  149  198  29  14  ...  30   6   3   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      4   4   1   3    +1   0  +1  
1      2   4   2   2     0   0   0  
2      1   0   0   0    +5  +4  +1  
3      2   4   1   3    +6  +7  -1  
4      3   4   2   2    +2  +1  +1  

[5 rows x 38 columns]

Icebox 1:12:59


Type        Name Team     R              ACS             K      ... HS%  FK  \
Side        Name Team   all   atk   def  all  atk  def all atk  ... def all   
0          trexx  KOI  1.24  1.72  0.77  304  391  218  27  18  ...  36   2   
1        sheydos  KOI  1.15  1.56  0.73  193  241  147  17  13  ...  28   1   
2         Wolfen  KOI  0.81  0.81  0.81  168  176  162  14   7  ...  50   5   
3         starxo  KOI  0.73  1.00  0.46  134  149  119  10   6  ...  20   0   
4     koldamenta  KOI  0.59  1.08  0.09   73  134   12   5   5  ...   0   0   

Type          FD         +/–.1          
Side atk def all atk def   all atk def  
0      1   1   1   1   0    +1   0  +1  
1      1   0   2   0   2    -1  +1  -2  
2      4   1   8   5   3    -3  -1  -2  
3      0   0   4   0   4    -4   0  -4  
4      0   0   1   0   1    -1   0  -1  

[5 rows x 38 columns]

...

Pearl 54:20


Type        Name Team     R              ACS             K      ... HS%  FK  \
Side        Name Team   all   atk   def  all  atk  def all atk  ... def all   
0     Sayaplayer   T1  0.83  0.50  1.05  196  103  259  14   3  ...  44   5   
1       Munchkin   T1  0.78  0.19  1.18  157   95  199  11   2  ...  19   0   
2          Carpe   T1  0.77  0.28  1.10  172   94  224  12   3  ...  30   0   
3            ban   T1  0.74  0.91  0.63  194  273  143  13   7  ...  19   2   
4           xeta   T1  0.71  0.70  0.71  172  229  135  10   5  ...  27   0   

Type          FD         +/–.1          
Side atk def all atk def   all atk def  
0      1   4   5   2   3     0  -1  +1  
1      0   0   5   2   3    -5  -2  -3  
2      0   0   0   0   0     0   0   0  
3      0   2   0   0   0    +2   0  +2  
4      0   0   3   3   0    -3  -3   0  

[5 rows x 38 columns]

Type    Name Team     R              ACS             K      ... HS%  FK      \
Side    Name Team   all   atk   def  all  atk  def all atk  ... def all atk   
0      Mazin  FUR  1.46  1.41  1.52  205  231  168  16  11  ...  13   1   0   
1     mwzera  FUR  1.38  1.24  1.60  268  233  322  18  10  ...   8   4   1   
2      dgzin  FUR  1.32  0.81  2.09  291  257  343  21  10  ...  25   4   3   
3      Quick  FUR  1.07  1.01  1.15  199  185  222  14   8  ...  35   2   1   
4     Khalil  FUR  1.05  0.87  1.32  155  153  159  11   5  ...  40   2   1   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      1   1   1   0     0  -1  +1  
1      3   0   0   0    +4  +1  +3  
2      1   4   4   0     0  -1  +1  
3      1   2   1   1     0   0   0  
4      1   0   0   0    +2  +1  +1  

[5 rows x 38 columns]

Icebox -


TypeError: expected string or bytes-like object, got 'int'

In [67]:
sample_match = completed_matches[21]
match_res = httpx.get(match_url := f'{base_url}{sample_match.attributes["href"]}')
print(httpx.get(match_url))

match_html = HTMLParser(match_res.text)

<Response [200 OK]>


In [68]:
maps = match_html.css('div.vm-stats-game')

# rearrange the map order
maps = [maps[1],maps[0],maps[2]]

# extract data of each map
for m in maps:
    # get the current map pick name or overall if it the match summary
    current_map = m.css_first('div.map')
    match_header = 'Overall' if not current_map else\
                    ' '.join(re_strip(' ',current_map.text()).split(' ')[::2])

    print(match_header)


    tables = m.css('table.wf-table-inset')

    for table in tables:
        df = pd.read_html(table.html)[0]
        display(extract_df(df))

Overall


Type        Name Team     R              ACS             K      ... HS%  FK  \
Side        Name Team   all   atk   def  all  atk  def all atk  ... def all   
0     Sayaplayer   T1  0.83  0.50  1.05  196  103  259  26   3  ...  44   5   
1       Munchkin   T1  0.78  0.19  1.18  157   95  199  18   2  ...  19   0   
2          Carpe   T1  0.77  0.28  1.10  172   94  224  29   3  ...  30   0   
3            ban   T1  0.74  0.91  0.63  194  273  143  16   7  ...  19   2   
4           xeta   T1  0.71  0.70  0.71  172  229  135  17   5  ...  27   0   

Type          FD         +/–.1          
Side atk def all atk def   all atk def  
0      1   4   5   2   3     0  -1  +1  
1      0   0   5   2   3    -5  -2  -3  
2      0   0   0   0   0     0   0   0  
3      0   2   0   0   0    +2   0  +2  
4      0   0   3   3   0    -3  -3   0  

[5 rows x 38 columns]

Type    Name Team     R              ACS             K      ... HS%  FK      \
Side    Name Team   all   atk   def  all  atk  def all atk  ... def all atk   
0      Mazin  FUR  1.46  1.41  1.52  205  231  168  35  11  ...  13   1   0   
1     mwzera  FUR  1.38  1.24  1.60  268  233  322  32  10  ...   8   4   1   
2      dgzin  FUR  1.32  0.81  2.09  291  257  343  37  10  ...  25   4   3   
3      Quick  FUR  1.07  1.01  1.15  199  185  222  35   8  ...  35   2   1   
4     Khalil  FUR  1.05  0.87  1.32  155  153  159  16   5  ...  40   2   1   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      1   1   1   0     0  -1  +1  
1      3   0   0   0    +4  +1  +3  
2      1   4   4   0     0  -1  +1  
3      1   2   1   1     0   0   0  
4      1   0   0   0    +2  +1  +1  

[5 rows x 38 columns]

Pearl 54:20


Type        Name Team     R              ACS             K      ... HS%  FK  \
Side        Name Team   all   atk   def  all  atk  def all atk  ... def all   
0     Sayaplayer   T1  0.83  0.50  1.05  196  103  259  14   3  ...  44   5   
1       Munchkin   T1  0.78  0.19  1.18  157   95  199  11   2  ...  19   0   
2          Carpe   T1  0.77  0.28  1.10  172   94  224  12   3  ...  30   0   
3            ban   T1  0.74  0.91  0.63  194  273  143  13   7  ...  19   2   
4           xeta   T1  0.71  0.70  0.71  172  229  135  10   5  ...  27   0   

Type          FD         +/–.1          
Side atk def all atk def   all atk def  
0      1   4   5   2   3     0  -1  +1  
1      0   0   5   2   3    -5  -2  -3  
2      0   0   0   0   0     0   0   0  
3      0   2   0   0   0    +2   0  +2  
4      0   0   3   3   0    -3  -3   0  

[5 rows x 38 columns]

Type    Name Team     R              ACS             K      ... HS%  FK      \
Side    Name Team   all   atk   def  all  atk  def all atk  ... def all atk   
0      Mazin  FUR  1.46  1.41  1.52  205  231  168  16  11  ...  13   1   0   
1     mwzera  FUR  1.38  1.24  1.60  268  233  322  18  10  ...   8   4   1   
2      dgzin  FUR  1.32  0.81  2.09  291  257  343  21  10  ...  25   4   3   
3      Quick  FUR  1.07  1.01  1.15  199  185  222  14   8  ...  35   2   1   
4     Khalil  FUR  1.05  0.87  1.32  155  153  159  11   5  ...  40   2   1   

Type      FD         +/–.1          
Side def all atk def   all atk def  
0      1   1   1   0     0  -1  +1  
1      3   0   0   0    +4  +1  +3  
2      1   4   4   0     0  -1  +1  
3      1   2   1   1     0   0   0  
4      1   0   0   0    +2  +1  +1  

[5 rows x 38 columns]

Icebox -


TypeError: expected string or bytes-like object, got 'int'

In [71]:
pd.read_html(table.html)

[      Unnamed: 0  Unnamed: 1   R  ACS   K         D  A  +/–  KAST  ADR  HS%  \
 0        xeta T1         NaN NaN  NaN   7  /  14  /  5   -7   NaN  NaN  NaN   
 1    Munchkin T1         NaN NaN  NaN   7  /  15  /  5   -8   NaN  NaN  NaN   
 2  Sayaplayer T1         NaN NaN  NaN  12  /  16  /  2   -4   NaN  NaN  NaN   
 3         ban T1         NaN NaN  NaN   3  /  17  /  1  -14   NaN  NaN  NaN   
 4       Carpe T1         NaN NaN  NaN  17  /  13  /  1    4   NaN  NaN  NaN   
 
    FK  FD  +/–.1  
 0 NaN NaN    NaN  
 1 NaN NaN    NaN  
 2 NaN NaN    NaN  
 3 NaN NaN    NaN  
 4 NaN NaN    NaN  ]

In [83]:
def extract_data(data) -> str:

    list_data = re.findall("[\d+-.]+",data)

    if len(list_data) < 3:
        result = '0 0 0'
    else:
        result = ' '.join(list_data)

    return result



df = pd.read_html(table.html)[0]
filled_df =df.fillna('Nan Nan Nan').astype('object')

name = filled_df.iloc[:,0].map(lambda s : s.split(' ')[0])
team = filled_df.iloc[:,0].map(lambda s : s.split(' ')[1])

filled_df.iloc[:,0] = name
filled_df.iloc[:,1] = team

formatted_df = filled_df.rename(columns={'Unnamed: 0':'Name','Unnamed: 1':'Team'})

formatted_df.iloc[:,2:] = formatted_df.iloc[:,2:].applymap(extract_data)

formatted_df

TypeError: expected string or bytes-like object, got 'int'

# OOP Design

In [7]:
class Event:
    def __init__(self,url):
        self.url = url
        # self.dates -> str(maybe pd.datatime)
        # self.players -> List[Player]
        # self.prize_pool -> int,float
        # self.region -> str
        # self.matches -> List[Match]

In [8]:
lock_in_url = 'https://www.vlr.gg/event/matches/1188/champions-tour-2023-lock-in-s-o-paulo/?series_id=all'


class Event_Slow:
    def __init__(self,url):
        self.url = url
        self.info = pd.DataFrame({
            'Name' : ['Dates','Prize Pool','Location'],
            'Value' : [self.dates(),self.prize_pool(),self.location()]
        })
        
    def page_html(self):
        res = httpx.get(self.url)
        
        return HTMLParser(res.text)
    
    def prize_pool(self):
        
        return self.page_html().css('div.event-desc-item-value')[1].text().strip()
    
    def location(self):
        return self.page_html().css_first('div.event-desc-item.mod-last')\
                        .css_first('div.event-desc-item-value').text().strip()
    def dates(self):
        dates = self.page_html().css_first('div.event-desc-item-value').text().strip().split(' - ')
        return pd.to_datetime(dates).strftime('%y/%m/%d')
    
    def date_range(self):
        return pd.date_range(start=self.dates()[0],end=self.dates()[1],freq='D')
    
        
        
        
        

In [9]:
lock_in = Event_Slow(lock_in_url)
lock_in.prize_pool()

'$500,000 USD'

In [53]:
class Event:
    def __init__(self,url:str) -> None:
        self.url = url
        self.html = HTMLParser(httpx.get(url).text)
        header = self.html.css('div.event-desc-item-value')
        
        self.info = pd.DataFrame({
            'dates' : header[0].text().strip(),
            'prize_pool' : header[1].text().strip(),
            'location' : header[2].text().strip()
        },index=['Info'])
        
        
        
    def get_matches(self):
        if hasattr(self,'matches'):
            return self.matches
        
        completed_matches = [Match(match) for match in self.html.css('a.wf-module-item')
                                if match.css_first('div.ml-status').text() == 'Completed']
        self.matches = completed_matches
        return completed_matches
        

In [61]:
class Match:
    def __init__(self,node) -> None:
        self.match_url = 'https://www.vlr.gg' + node.attributes['href']
        self.html = HTMLParser(httpx.get(self.match_url).text)
        
        match_name = self.html.css_first('title').text().strip().split(' | ')[0].split(' vs. ')
        
        messy_match_result = self.html.css_first('div.js-spoiler').text()
        match_result = ' ' + re_strip('',messy_match_result) + ' '
        
        self.match_sum = match_result.join(match_name)
        
        self.stage = re_strip(' ',self.html.css_first('div.match-header-event-series').text())
        
        self.ban_pick = re_strip(' ',self.html.css_first('div.match-header-note').text()).split('; ')
        
        
    def __repr__(self):
        return self.match_sum
        
        
        
        
        
        

In [62]:
lock_in = Event(lock_in_url)

In [63]:
lock_in.get_matches()

AttributeError: 'NoneType' object has no attribute 'text'

In [57]:
lock_in.matches

[KOI 0:2 NRG Esports,
 DetonatioN FocusMe 0:2 Giants Gaming,
 FunPlus Phoenix 1:2 Karmine Corp,
 BBL Esports 1:2 DRX,
 Cloud9 2:0 Paper Rex,
 Team Heretics 0:2 Evil Geniuses,
 MIBR 0:2 Talon Esports,
 Gen.G 0:2 LOUD,
 NRG Esports 2:1 Giants Gaming,
 LOUD 2:0 Karmine Corp,
 DRX 2:1 Cloud9,
 Evil Geniuses 0:2 Talon Esports,
 NRG Esports 1:2 LOUD,
 DRX 2:1 Talon Esports,
 Team Liquid 0:2 Team Secret,
 Natus Vincere 2:0 KRÜ Esports,
 ZETA DIVISION 0:2 Leviatán,
 Team Vitality 2:1 Global Esports,
 FUT Esports 2:0 Rex Regum Qeon,
 100 Thieves 2:1 EDward Gaming,
 Sentinels 0:2 FNATIC,
 T1 0:2 FURIA,
 Team Secret 0:2 Natus Vincere,
 Leviatán 2:0 Team Vitality,
 FUT Esports 1:2 100 Thieves,
 FNATIC 2:0 FURIA,
 Natus Vincere 2:0 Leviatán,
 100 Thieves 0:2 FNATIC,
 LOUD 3:2 DRX,
 Natus Vincere 0:3 FNATIC,
 Team tarik 0:1 Team FRTTT,
 LOUD 2:3 FNATIC]

In [59]:
lock_in.matches[2].stage

'Bracket Stage: Alpha - Round of 16'